In [1]:
%load_ext autoreload 
%autoreload 2

Digitized data from Planert 2013, Fig. 4 B & D.

![image](planert2013/Planert2013-Fig4-DA-pone.0057054.g004.png)

Reference: 

Planert H, Berger TK, Silberberg G. Membrane properties of striatal direct and indirect pathway neurons in mouse and rat slices and their modulation by dopamine. PLoS One. 2013;8(3):e57054. doi:10.1371/journal.pone.0057054

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'
pio.templates.default = 'seaborn'

In [3]:
def postprocess(filename):
    raw_data = np.genfromtxt(filename, delimiter=',')
    pre = raw_data[::2]
    post = raw_data[1::2]
    
    assert post.shape == pre.shape, f"length mismatch ({post.shape=}) != ({pre.shape=})"

    slopes = (post[:,1] - pre[:,1]) / (post[:,0] - pre[:,0])
    pre_y = post[:,1]-slopes
    pre_x = np.zeros(pre_y.shape)

    post_x = post[:,0]
    post_y = post[:,1]

    return pre_x, pre_y, post_x, post_y

In [4]:
(df['pre'] - df['pre'].round(0).astype(int))

type: name 'df' is not defined

In [20]:
table_data = {'post' : [], 'pre' : [], 'holding': [], 'receptor': [], 'filename': []}

for filename, holding, receptor in [('planert2013/d1-60mV.csv', -60, 'D1'),
                                    ('planert2013/d1-80mV.csv', -80, 'D1'),
                                    ('planert2013/d2-60mV.csv', -60, 'D2'),
                                    ('planert2013/d2-80mV.csv', -80, 'D2')]:
    print(f"Processing {filename}...")
    pre_x, pre_y, post_x, post_y = postprocess(filename)
    table_data['post'] += list(post_y)
    table_data['pre']  += list(pre_y)
    table_data['holding']  += [holding] * len(pre_y)
    table_data['receptor']  += [receptor] * len(pre_y)
    table_data['filename']  += [filename] * len(pre_y)

df = pd.DataFrame(table_data)
df['change'] = (df['post'] - df['pre']) / df['pre']
df['experiment'] = df['receptor'] + '-' + df['holding'].astype(str)

# The figure shows spike counts so we round the digitized values
# to the nearest integer
df['i_pre'] = df['pre'].round(0).astype(int)
df['i_post'] = df['post'].round(0).astype(int)
df['i_change%'] = ((df['i_post'] - df['i_pre']) / df['i_pre'])*100

# check rounding error
df['pre_diff'] = (df['pre'] - df['i_pre']).abs()
df['post_diff'] = (df['post'] - df['i_post']).abs()

Processing planert2013/d1-60mV.csv...
Processing planert2013/d1-80mV.csv...
Processing planert2013/d2-60mV.csv...
Processing planert2013/d2-80mV.csv...


In [23]:
px.scatter(df, y='i_change%', x='experiment', color='experiment')

In [24]:
px.box(df, y='i_change%', x='experiment', color='experiment')

In [8]:
px.box(df, y='post', x='experiment', color='experiment')

In [9]:
def plot(df, receptor, holding):
    _df = df.loc[(df['receptor'] == receptor) & (df['holding'] == holding)]
    
    fig = go.Figure()

    for _, row in _df.iterrows():
        fig.add_trace(go.Scatter(x=['pre','post'], y=[row['i_pre'], row['i_post']]))

    fn = _df['filename'].unique()
    fig.update_layout(title=f'Files: {fn}', height=500, width=500)
    
    return fig
    




In [10]:
plot(df, 'D1', -60)

In [11]:
plot(df, 'D2', -60)

In [12]:
plot(df, 'D1', -80)

In [13]:
plot(df, 'D2', -80)

In [21]:
# points with rounding differences larger than 0.4 were manually checked. 
# Two of these were adjusted by hand in the .csv files after manual inspection.
df.sort_values('post_diff')[::-1]


,post,pre,holding,receptor,filename,change,experiment,i_pre,i_post,i_change%,pre_diff,post_diff
26,4.342656,3.185961,-80,D1,planert2013/d1-80mV.csv,0.363060,D1--80,3,4,33.333333,0.185961,0.342656
23,13.290029,1.896696,-80,D1,planert2013/d1-80mV.csv,6.006938,D1--80,2,13,550.000000,0.103304,0.290029
25,8.278854,1.896872,-80,D1,planert2013/d1-80mV.csv,3.364476,D1--80,2,8,300.000000,0.103128,0.278854
24,9.271253,2.159769,-80,D1,planert2013/d1-80mV.csv,3.292707,D1--80,2,9,350.000000,0.159769,0.271253
20,3.246393,6.125866,-80,D1,planert2013/d1-80mV.csv,-0.470052,D1--80,6,3,-50.000000,0.125866,0.246393
27,4.229984,2.227514,-80,D1,planert2013/d1-80mV.csv,0.898970,D1--80,2,4,100.000000,0.227514,0.229984
17,4.226484,1.437832,-80,D1,planert2013/d1-80mV.csv,1.939485,D1--80,1,4,300.000000,0.437832,0.226484
16,3.223744,1.119396,-80,D1,planert2013/d1-80mV.csv,1.879895,D1--80,1,3,200.000000,0.119396,0.223744
18,6.221388,1.237912,-80,D1,planert2013/d1-80mV.csv,4.025710,D1--80,1,6,500.000000,0.237912,0.221388
39,11.168033,4.000000,-80,D2,planert2013/d2-80mV.csv,1.792008,D2--80,4,11,175.000000,0.000000,0.168033
